In [1]:
!guardrails hub install hub://guardrails/provenance_embeddings --quiet
!guardrails hub install hub://guardrails/provenance_llm --quiet

Installing hub://guardrails/provenance_embeddings...
ERROR:guardrails-cli:No module named 'guardrails.hub.guardrails.provenance_embeddings'
Installing hub://guardrails/provenance_llm...
ERROR:guardrails-cli:No module named 'guardrails.hub.guardrails.provenance_llm'


# Adding Provenance guardrails to the workflow


As we've seen, LLMs are great at generating structured data. Adding guardrails to that workflow gives us tighter responses and more control. While structured data is super useful as an interface between different systems, it's not always the best way to communicate with humans. For that, we need to generate unstructured text.

A problem with generating unstructured text is hallucination. Hallucination occurs when an LLM generates text that is not supported by the input.

Guardrails has provenance validators that detect and limit hallucinations. In this example, we show how to set them up.

**Both provenance validators support validating LLM-generated text on both sentence-level and entire text level. In this demo, we'll demo how sentence-level validation works. In sentence-level validaton, each sentence in the LLM-generated text will be checked for provenance.**


## ProvenanceEmbeddings

When you need to confirm that an LLM response is supported by a source text, you can use the ```ProvenanceEmbeddings``` Guardrails validator function. This function takes a list source articles, an LLM response (or prompt), and a threshold. It validates whether the response is "close" enough to the source articles. The threshold is a number between 0 and 1, where 1 means the response is not close to the source articles, and 0 means the response is very close to the source articles.

The provenance validator function can also wrap an LLM response (similar to what we saw in the first example). It can also wrap a query function to interact with your vector database as a source. These two uses are out-of-scope for this document.

For the sake of this post, let's use the following configuration for our validator:

1. `sources`: List of strings, each representing a source.
2. `embed function`: A function that takes a list of strings of length k and returns a 2D numpy array of shape (k, embedding_dim).

## Set up our source data


In [2]:
sources = [
    """1 Move the litter box to the right location. Cats may stop using the litter box after a scary experience in the
area, such as a loud noise or harassment by another pet. [2] They may also dislike the spot you chose after moving
the litter box, or moving to a new home. Keep the litter box in a quiet, low-traffic spot where the cat can see
people coming. Choose a room with at least two exits so the cat doesn't feel cornered. [3] Keep litter boxes away
from food and water bowls. Cats do not like to combine these two areas. Signs that your cat may have had an
unpleasant experience in the litter box including running quickly in and out of the litter box, or using an area
near the litter box. [4] Try moving the box to a new room if you notice this. Keep at least one litter box on every
floor of a multistory home. [5] Keep litter boxes away from food and water bowls. Cats do not like to combine these
two areas. Signs that your cat may have had an unpleasant experience in the litter box including running quickly in
and out of the litter box, or using an area near the litter box. [4] Try moving the box to a new room if you notice
this. Keep at least one litter box on every floor of a multistory home. [5] 2 Play with toys near the litter box.
Play with your cat in the same general area as the litter box. Leave toys (but not food) in the room so the cat
spends time there and develops positive associations. [6] You can bring the cat to the litter box to investigate on
its own, but do not drop it inside or reward it with treats for using it. These tactics can backfire by making the
cat uncomfortable or afraid. [7] Unlike dogs, cats should choose the litter box on their own, especially if they
used one in the past. You can bring the cat to the litter box to investigate on its own, but do not drop it inside
or reward it with treats for using it. These tactics can backfire by making the cat uncomfortable or afraid. [7]
Unlike dogs, cats should choose the litter box on their own, especially if they used one in the past. 3 Keep the
litter box clean. [8] If your cat perches on the edge of the box or eliminates right next to it, the box might be
too dirty for it. Remove clumps and top up with fresh litter at least once a day, preferably twice. Rinse the
litter box once a week with baking soda or unscented soap. [9] If you use non-clumping litter, change the whole box
every couple days to prevent odor build up, which can drive away the cat. [10] Do not clean the litter box with
scented products. Do not use a disinfectant unless it is specifically made for litter boxes, as many of them
contain chemicals toxic to cats. [11] If you use non-clumping litter, change the whole box every couple days to
prevent odor build up, which can drive away the cat. [10] Do not clean the litter box with scented products. Do not
use a disinfectant unless it is specifically made for litter boxes, as many of them contain chemicals toxic to
cats. [11] 4 Switch to new litter gradually. If you bought a different kind of litter, introduce it slowly. Mix a
little of it in with the old type, and gradually increase the proportion each time you change the litter box. [12]
Cats usually find it easier to adjust to unscented litter with a similar texture to their old litter. [13] If the
old type of litter is no longer available, buy two or three new types. Put them in separate litter boxes side by
side and let the cat choose its favorite. Try adjusting the depth of the litter, especially if it has a different
texture than the cat is used to. Many cats prefer a shallow layer of litter, less than two inches (5 cm.) deep.
Long-haired cats often like an extra-shallow layer so they can dig to the floor of the box. [14] If the old type of
litter is no longer available, buy two or three new types. Put them in separate litter boxes side by side and let
the cat choose its favorite. Try adjusting the depth of the litter, especially if it has a different texture than
the cat is used to. Many cats prefer a shallow layer of litter, less than two inches (5 cm.) deep. Long-haired cats
often like an extra-shallow layer so they can dig to the floor of the box. [14] 5 Troubleshoot new litter boxes. If
your cat hasn't responded well to a recent litter box replacement, try these adjustments to make it more appealing:
[15] Some cats prefer covered boxes, and other prefer open trays. Try adding or removing the hood. Remove plastic
liners from the litter box. These can snag a cat's claws. [16] Most cats adjust well to self-cleaning litter boxes
but not all. There is a risk of an anxious cat being frightened by the motor, and refusing to use the box as a
result. If in doubt it’s best to stick with a regular litter box. If the box is smaller than the old one, you
probably need to replace it with something larger. A large box with low sides works best; some people use a plastic
sweater box. [17] Some cats prefer covered boxes, and other prefer open trays. Try adding or removing the hood.
Remove plastic liners from the litter box. These can snag a cat's claws. [16] Most cats adjust well to
self-cleaning litter boxes but not all. There is a risk of an anxious cat being frightened by the motor, and
refusing to use the box as a result. If in doubt it’s best to stick with a regular litter box. If the box is
smaller than the old one, you probably need to replace it with something larger. A large box with low sides works
best; some people use a plastic sweater box. [17] 6 Clean up urine and feces with an enzymatic cleaner. When the
cat fails to use its litter box, clean the area with an enzymatic cleaner designed for cat urine (or a 10% solution
of enzymatic washing powder in water). Rinse with cold water. This removes the smell of urine that can attract a
cat to return to the same location. [18] For best results, lightly spray the area with rubbing alcohol after the
water has dried. Scrub gently and let it air dry. For best results, lightly spray the area with rubbing alcohol
after the water has dried. Scrub gently and let it air dry. 7 Make soiled areas less appealing. If your cat has
developed a habit of going in certain locations, block its access to those areas, or find temporary ways to
discourage it until it has relearned good habits: If the cat uses a dark hiding spot, install a bright light,
preferably motion-activated. [19] Make rugs or other areas unpleasant to stand on with tin foil or double-sided
tape.""",
    """[20] If the cat urinated on curtains, pin them up out of reach until the cat is back to using its litter.
[21] Cover targeted furniture with plastic sheets or shower curtains. [22] Fill bathtubs and sinks with a shallow
layer of water when not in use. [23] If the cat uses a dark hiding spot, install a bright light, preferably
motion-activated. [19] Make rugs or other areas unpleasant to stand on with tin foil or double-sided tape. [20] If
the cat urinated on curtains, pin them up out of reach until the cat is back to using its litter. [21] Cover
targeted furniture with plastic sheets or shower curtains. [22] Fill bathtubs and sinks with a shallow layer of
water when not in use. [23] 8 Place litter boxes in the problem areas. One solution is to give in to your cat's
preferences and add new boxes in the areas it's using as a bathroom. Clearly this isn't ideal if your cat is using
the center of the living room rug, but it's worth considering if the cat is soiling an out-of-the-way corner of
your house. Another option is to move the cat's food bowl to this location. Most cats will not eliminate and eat in
the same place. Another option is to move the cat's food bowl to this location. Most cats will not eliminate and
eat in the same place. 9 Use your cat's preference to your advantage. If none of these adjustments work, you may
have to resort to a slower transition. For example, if your cat prefers to urinate on carpet, place a piece of
similar carpet in the litter box. If the cat is willing to use that option, add a small amount of litter on top of
the carpet the next day. Continue adding more litter and replacing the carpet when it gets too dirty, until the cat
is transitioned to litter completely. You may have to confine the cat to a non-carpeted area of the house for a
while to get this option to work, or to roll up your carpets temporarily. Confinement can backfire if the cat is
stressed or bored. Similarly, if your cat is indoor/outdoor, or if your cat used to eliminate mostly outside, add
garden soil or sand (without any fertilizer) to the box. Again, transition slowly from sand/soil to litter by
adding small amounts of the new substrate to the preferred substrate a little at a time. You may have to confine
the cat to a non-carpeted area of the house for a while to get this option to work, or to roll up your carpets
temporarily. Confinement can backfire if the cat is stressed or bored. Similarly, if your cat is indoor/outdoor, or
if your cat used to eliminate mostly outside, add garden soil or sand (without any fertilizer) to the box. Again,
transition slowly from sand/soil to litter by adding small amounts of the new substrate to the preferred substrate
a little at a time. Method 2 1 Spay or neuter your cat. This is not a requirement for litter training, but it does
make urination outside the litter box much less likely. Unneutered males are especially likely to spray urine when
they're stressed, not getting along with another male, or demonstrating their availability to a female cat. [24]
The sooner this happens, the more likely the behavior will stop. If it goes on too long, the habit can persist even
after surgery. [25] The sooner this happens, the more likely the behavior will stop. If it goes on too long, the
habit can persist even after surgery. [25] 2 Reduce stress in your cat . [26] Just like humans, cats may experience
stress from changes in their environment or schedule. Your cat may stop using the litterbox after a person or other
animal leaves the household, or when a new one moves in. Some cats even respond badly to redecoration. Here are a
few ways to help: Provide private places where the cat can be on its own, including hiding spots and high perches.
If your cat is allowed outside, let it come and go whenever it likes. [27] Let your cat initiate contact, and be
calm and consistent in your response. Some cats are stressed because they aren't getting enough playtime, while
others dislike being pet or picked up whenever the owner feels like it. If the cat's behavior continues, consult a
veterinarian or animal behaviorist. Provide private places where the cat can be on its own, including hiding spots
and high perches. If your cat is allowed outside, let it come and go whenever it likes. [27] Let your cat initiate
contact, and be calm and consistent in your response. Some cats are stressed because they aren't getting enough
playtime, while others dislike being pet or picked up whenever the owner feels like it. If the cat's behavior
continues, consult a veterinarian or animal behaviorist. 3 Respond to vertical spraying. If your cat backs up
against a vertical surface, wiggles its tail, and releases a spray of urine, your cat is spraying. If you don't see
it in action, look for roundish areas of strong-smelling urine a little higher up than the height of your cat's
rear end, with streak marks running down to the baseboard or floor. Any cat can perform this territorial behavior,
but it is most common in unneutered, male cats. Here's how to respond if your cat is spraying: Spraying is often a
response to stress or the presence of other cats. [28] Follow the advice above to address this. Spraying can be a
response to a new neighborhood cat, especially if the spray is focused on a door, window, or air vent. Try to keep
the cat out of your yard , or close the blinds so your cat can't see it. About 30% of cats that vets examine for
spraying have a medical condition. [29] It's a good idea to have your cat examined, especially if you cannot find a
solution. Spraying is often a response to stress or the presence of other cats. [28] Follow the advice above to
address this. Spraying can be a response to a new neighborhood cat, especially if the spray is focused on a door,
window, or air vent. Try to keep the cat out of your yard , or close the blinds so your cat can't see it. About 30%
of cats that vets examine for spraying have a medical condition. [29] It's a good idea to have your cat examined,
especially if you cannot find a solution. 4 Replace small boxes as a kitten grows. If you adopted your cat as a
kitten, it may need a larger litter box once it grows up. The cat should be able to turn around comfortably, and
still be able to find a clean spot if you miss a cleanup. [30] Cats dislike change, and may take a while to adapt
to the new box. Follow the instructions above if problems continue. Cats dislike change, and may take a while to
adapt to the new box. Follow the instructions above if problems continue. 5 Clip away matted fur in long-haired
cats . Some long-haired cats dirty the fur around their rear when they eliminate. This can cause painful or
unpleasant experiences that the cat learns to associate with the litter box. If you notice this happening,
carefully clip away matted fur from the area. [31] 6 Minimize damage when the owner is away. Some cats react poorly
when their owner is away. They may try to urinate somewhere with a strong scent of the owner, usually the bed.
Instruct the pet sitter to keep the bedroom door shut, and provide extra litter boxes so the cat can always reach
one without walking by the pet sitter. [32] If possible, hire a pet sitter the cat already knows, or at least
introduce them before you leave. If possible, hire a pet sitter the cat already knows, or at least introduce them
before you leave. 7 Improve behavior in multi-pet households. Urine marking is a common reaction to conflict with
another cat or dog, which can happen even if the animals got along in the past. For best results, make sure each
animal can access resources without approaching the other: [33] Keep one litter box for each animal, plus one
extra. Put each one in a separate location with at least two exit routes if possible. Give each animal its own bed
and food bowl. Keep these resources away from litter boxes and from each other. Provide plenty of perches and
hiding spaces for each cat. Keep one litter box for each animal, plus one extra. Put each one in a separate
location with at least two exit routes if possible. Give each animal its own bed and food bowl. Keep these
resources away from litter boxes and from each other. Provide plenty of perches and hiding spaces for each cat. 8
Separate animals if bad behavior continues. If your cat still won't use the litter box, or still acts aggressive to
the other animal, try a stricter method of separation. This is often necessary when bringing a new cat into the
household: [34] Separate the cats into rooms with a closed door in between, so they do not see each other. Gather
each cat's facial pheromones (forehead, cheeks, chin) on a small sock or gauze pad and put each sock in the others'
space. Then, brush each cat to collect their facial pheromones and place one cat's pheromones on the other,
creating a group scent. Swap spaces so they can smell each other's scent in the environment. Introduce them
carefully, providing a high place and hidey holes for escape if one is aggressive to the other. Consult a cat
behaviorist if the cats simply can't get along. Separate the cats into rooms with a closed door in between, so they
do not see each other. Gather each cat's facial pheromones (forehead, cheeks, chin) on a small sock or gauze pad
and put each sock in the others' space. Then, brush each cat to collect their facial pheromones and place one cat's
pheromones on the other, creating a group scent. Swap spaces so they can smell each other's scent in the
environment. Introduce them carefully, providing a high place and hidey holes for escape if one is aggressive to
the other. Consult a cat behaviorist if the cats simply can't get along. Method 3 1 Assess whether your cat is
having a hard time urinating. If you see your cat straining while trying to urinate, or spending a long time trying
with no result, take your cat to the vet immediately. Male cats in particular can develop urinary conditions where
the urethra (the tube from the bladder to the penis) narrows or gets blocked. Typically small amounts of urine can
pass until the cat is completely blocked and cannot urinate at all. This is a life threatening situation and needs
immediate medical care. Blockages can also occur in the intestinal tract. Some cats with a urinary infection or
blockage spend a lot of time trying to urinate, licking their genitals, or crying at their owner. [35] Some cats
with a urinary infection or blockage spend a lot of time trying to urinate, licking their genitals, or crying at
their owner.""",
    """[35] 2 Evaluate whether your cat is having fecal issues. Fecal constipation does occur in cats and can
lead to chronic issues requiring special diets and laxatives. Diarrhea is not uncommon either, including chronic
diarrhea related to inflammatory bowel disease. Any of these conditions are uncomfortable for the cat and may cause
the cat to be afraid of the litter box or unable to get to the litter box in time to avoid an accident. Many cats
with inflammatory bowel disease only have occasional symptoms for no obvious reason. Appetite changes, lethargy,
vomiting, or increased hairball production can be signs of intestinal discomfort. Many cats with inflammatory bowel
disease only have occasional symptoms for no obvious reason. Appetite changes, lethargy, vomiting, or increased
hairball production can be signs of intestinal discomfort. 3 Improve physical access to the box. If your cat is
older or has an injury, it may not be able to get into the litter box as comfortably as it once could. Does your
cat limp, need help jumping up to a chair or bed, have episodes of shaking legs, or seem painful around its spine
or tail? If so, take the cat to the vet immediately! You may be able to make the cat more comfortable by getting a
litter box with lower sides, or with a "doorway" cut into the side. You may need to consider a larger box so that
the cat can turn around easily once inside. An overweight cat may not fit comfortably in the box any longer. Get a
larger box and place the cat on a diet. Consult with your veterinarian about safe weight loss strategies for your
cat. An overweight cat may not fit comfortably in the box any longer. Get a larger box and place the cat on a diet.
Consult with your veterinarian about safe weight loss strategies for your cat. 4 Talk to your veterinarian to rule
out other medical problems. [36] Urinating outside of the litter box often results from a feline medical issue,
including urinary tract infections, diabetes mellitus, chronic renal disease, hyperthyroidism, bladder inflammation
with or without urinary crystals, and intestinal issues (IBD). When in doubt, have a veterinarian conduct a medical
checkup. Observe your cat so you are prepared to answer your veterinarian's questions. Possible questions include:
Does the cat urinate right by the litter box or further away? How large is the spot of urine? Does the cat ever try
to urinate in the litter box? Does the cat vocalize when urinating? Is there an increase in water consumption? Can
you tell if the urine is almost clear, normal color, or dark? How frequently is the cat urinating? Even if there is
no medical cause, your vet could prescribe anti-anxiety drugs to prevent spraying. This is not a guaranteed or
risk-free solution, so talk to your vet about the pros and cons. Observe your cat so you are prepared to answer
your veterinarian's questions. Possible questions include: Does the cat urinate right by the litter box or further
away? How large is the spot of urine? Does the cat ever try to urinate in the litter box? Does the cat vocalize
when urinating? Is there an increase in water consumption? Can you tell if the urine is almost clear, normal color,
or dark? How frequently is the cat urinating? Even if there is no medical cause, your vet could prescribe
anti-anxiety drugs to prevent spraying. This is not a guaranteed or risk-free solution, so talk to your vet about
the pros and cons. There are several possible reasons for this behavior. Your cat may have a medical condition that
needs treatment from a vet. However, it's also possible that your cat is afraid of the box because something scary
happened near it, like a loud sound or a fight with another household cat. It could also be as simple as the box is
dirty. I would first try to figure out what's causing your cat's issue. Most cats stop using their litter box
because they don't like the box, its location, the litter, or it's not clean enough for them. I would experiment
until you find what works best for your cat. If your cat still isn't using the litter box, talk to your vet to rule
out a medical issue. Cats are clean creatures that dislike soiling their living quarters. You can litter train a
stray but it does take patience. Keep the cat in a small room with a tray in one corner, food and water in another,
and a bed in a different corner. Given a choice between soiling their bed or food, the cat will most likely chose
the tray.""",
]

**Create embed function**

This embed funtion is a wrapper around the Cohere embed API. The function takes in a parameter called "text" and embeds that text. The embed function is used both for the source text as well as the output text. For the source text, the embed function is called only after Guardrails automatically chunks the text into smaller pieces. All the chunks are arranged in a list which is passed to Cohere. For the output text, the embed function is called, and then vector similarity is calculated between the source and output embeddings.


In [4]:
# Create an embedding function that uses a cohere model to embed the text
import cohere
import numpy as np
from guardrails import Guard
from guardrails.hub import ProvenanceEmbeddings
from typing import List, Union
import os

api_key = os.environ["COHERE_API_KEY"]

# Create a cohere client
cohere_client = cohere.Client(api_key=api_key)


def embed_function(text: Union[str, List[str]]) -> np.ndarray:
    """Embed the text using cohere's small model."""
    if isinstance(text, str):  # If text is a string, wrap it in a list
        text = [text]

    response = cohere_client.embed(
        model="embed-english-light-v2.0",
        texts=text,
    )
    embeddings_list = response.embeddings
    return np.array(embeddings_list)

Set up the rail with the ```ProvenanceEmbeddings``` validator on a single string output. The threshold set for this example is 0.3, which is more sensitive than usual. In this example, since some of the passed-in text is strongly supported by the source text, distance for those chunks is small and the result remains unchanged by Guardrails. However, some other information is not directly supporteds by the source. Guardrails took the Raw LLM output and removed the areas it deemed hallucinated.


In [5]:
# Initialize the guard object
guard = Guard().use(
    ProvenanceEmbeddings(threshold=0.3, validation_method="sentence", on_fail="fix")
)

Here, we show Guardrails fixes mocked out responses that hallucinated extra information that was not relevant or supported by the source text. Guardrails took the raw LLM outputs and removed the areas it deemed hallucinated.


In [6]:
# Use .parse directly when we already have the LLM output
guard.parse(
    llm_output="To retrain a cat to use the litter box, put its litter box in a low traffic area with at least 2 exits so the cat doesn’t feel cornered. Find the right litterbox for your cat, as well as the right litter. Place the litterbox in a good spot, away from heavily trafficked and noisy areas. Keep the litterbox very clean, and do not punish the cat or confine her to just one room. Once training is complete, it is important to clean the litter box regularly and to provide a safe and healthy environment for the cat.",
    metadata={"embed_function": embed_function, "sources": sources},
)

# See the guard logs
guard.history.last.tree

/Users/dtam/dev/guardrails/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭─────────────────────────────────────────────── Messages ────────────────────────────────────────────────╮ │
    │ │ No messages.                                                                                            │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ To retrain a cat to use the litter box, put its litter box in a low traffic area with at least 2 exits  │ │
    │ │ so the cat doesn’t feel cornered. Find the right litterbox for your cat, as well as the right litter.   │ │
    │ │ Place the litterbox in a good spot, away from heavily trafficked and noisy areas. Keep the litterbox    │ │
    │ │ very clean, and do not punish the cat or confine her to just one room. Once training is complete, it is │ │
    │ │ important to clean the litter box regularly and to provide a safe and healthy environment for the cat.  │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ 'To retrain a cat to use the litter box, put its litter box in a low traffic area with at least 2 exits │ │
    │ │ so the cat doesn’t feel cornered.\nFind the right litterbox for your cat, as well as the right          │ │
    │ │ litter.\nKeep the litterbox very clean, and do not punish the cat or confine her to just one            │ │
    │ │ room.\nOnce training is complete, it is important to clean the litter box regularly and to provide a    │ │
    │ │ safe and healthy environment for the cat.'                                                              │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

As you can see here, one of the sentences was removed from the output, as it wasn't deemed relevant.


In [7]:
# FAIL SCENARIO
guardrail_output = guard.parse(
    llm_output="Cats love lollipops and other sweet and sour foods. Cats can be retrained to use a litter box. Reward them when they do with lots of candy.",
    metadata={"embed_function": embed_function, "sources": sources},
)

guard.history.last.tree

/Users/dtam/dev/guardrails/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭─────────────────────────────────────────────── Messages ────────────────────────────────────────────────╮ │
    │ │ No messages.                                                                                            │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ Cats love lollipops and other sweet and sour foods. Cats can be retrained to use a litter box. Reward   │ │
    │ │ them when they do with lots of candy.                                                                   │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ 'Cats can be retrained to use a litter box.'                                                            │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Similarly, done here. Clearly, the other sentences were not relevant to the `sources`


## ProvenanceLLM

Guardrails also provides a ```ProvenanceLLM``` validator that uses evaluates provenance for LLM-generated text using an LLM. Currently, to perform this self-evaluation-based provenance check, you can pass in a name of any OpenAI ChatCompletion model like `gpt-3.5-turbo` or `gpt-4`, or pass in a callable that handles LLM calls. This callable can use any LLM, that you define. For simplicity purposes, we show here a demo of using OpenAI's `gpt-3.5-turbo` model.

To use the OpenAI API, you have 3 options:

- Set the OPENAI_API_KEY environment variable: os.environ["OPENAI_API_KEY"] = "[OpenAI_API_KEY]"
- Set the OPENAI_API_KEY using openai.api_key="[OpenAI_API_KEY]"
- Pass the api_key as a parameter to the parse function as done below, in this example

In [8]:
# Initialize the guard object
from guardrails.hub import ProvenanceLLM

guard_1 = Guard.for_string(
    validators=[
        ProvenanceLLM(
            validation_method="sentence",  # can be "sentence" or "full"
            llm_callable="gpt-3.5-turbo",  # as explained above
            top_k=3,  # number of chunks to retrieve
            on_fail="fix",
        )
    ],
    description="testmeout",
)

In [11]:
# Use .parse directly when we already have the LLM output
guard_1.parse(
    llm_output="To retrain a cat to use the litter box, put its litter box in a low traffic area with at least 2 exits so the cat doesn’t feel cornered. Find the right litterbox for your cat, as well as the right litter. Place the litterbox in a good spot, away from heavily trafficked and noisy areas. Keep the litterbox very clean, and do not punish the cat or confine her to just one room. Once training is complete, it is important to clean the litter box regularly and to provide a safe and healthy environment for the cat.",
    metadata={"embed_function": embed_function, "sources": sources},
    api_key=os.environ[
        "OPENAI_API_KEY"
    ],  # OpenAI API key if using an OpenAI model for 'llm_callable', OR using one of the above options
)

# See the guard logs
guard_1.history.last.tree

/Users/dtam/dev/guardrails/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(
/Users/dtam/dev/guardrails/guardrails/hub/guardrails/provenance_llm/validator/main.py:227: UserWarning: The LLM returned an invalid response. Considering the sentence as unsupported...
  warn(


Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭─────────────────────────────────────────────── Messages ────────────────────────────────────────────────╮ │
    │ │ No messages.                                                                                            │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ To retrain a cat to use the litter box, put its litter box in a low traffic area with at least 2 exits  │ │
    │ │ so the cat doesn’t feel cornered. Find the right litterbox for your cat, as well as the right litter.   │ │
    │ │ Place the litterbox in a good spot, away from heavily trafficked and noisy areas. Keep the litterbox    │ │
    │ │ very clean, and do not punish the cat or confine her to just one room. Once training is complete, it is │ │
    │ │ important to clean the litter box regularly and to provide a safe and healthy environment for the cat.  │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ ''                                                                                                      │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [10]:
# FAIL SCENARIO
guardrail_output = guard_1.parse(
    llm_output="Cats love lollipops and other sweet and sour foods. Cats can be retrained to use a litter box. Reward them when they do with lots of candy.",
    metadata={"embed_function": embed_function, "sources": sources},
    api_key=os.environ["OPENAI_API_KEY"],
)

guard_1.history.last.tree

/Users/dtam/dev/guardrails/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(
/Users/dtam/dev/guardrails/guardrails/hub/guardrails/provenance_llm/validator/main.py:227: UserWarning: The LLM returned an invalid response. Considering the sentence as unsupported...
  warn(


Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭─────────────────────────────────────────────── Messages ────────────────────────────────────────────────╮ │
    │ │ No messages.                                                                                            │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ Cats love lollipops and other sweet and sour foods. Cats can be retrained to use a litter box. Reward   │ │
    │ │ them when they do with lots of candy.                                                                   │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ ''                                                                                                      │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Similar outputs were observed with both validators. But, as with all LLMs, the validated outputs will change due to their stochastic nature. This demo also highlights how one can use an embedding model from Cohere along with an OpenAI LLM to get provenance.
